In [2]:

%pip install Flask Flask-SQLAlchemy Flask-JWT-Extended stripe


Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ------ --------------------------------- 0.3/1.7 MB ? eta -:--:--
   ------ --------------------------------- 0.3/1.7 MB ? eta -:--:--
   ------ --------------------------------- 0.3/1.7 MB ? eta -:--:--
   ------------ --------------------------- 0.5/1.7 MB 383.3 kB/s eta 0:00:03
   ------------ --------------------------- 0.5/1.7 MB 383.3 kB/s eta 0:00:03
   ------------ --------------------------- 0.5/1.7 MB 383.3 kB/s eta 0:00:03
   ------------------- -------------------- 0.8/1.7 MB 398.6 kB/s eta 0:00:03
   ------------------- -------------------- 0.8/1.7 MB 398.6 kB/s eta 0:00:03
   ------------------- -------------------- 0.8/1

In [7]:
from datetime import datetime, timedelta
import os
from flask import Flask, jsonify, request
from flask_sqlalchemy import SQLAlchemy
from flask_jwt_extended import (
    JWTManager, create_access_token, jwt_required, get_jwt_identity
)
import stripe
from werkzeug.security import generate_password_hash, check_password_hash
from threading import Thread

# ----------------------
# Configuration
# ----------------------
DATABASE_FILE = "ecommerce.db"
STRIPE_API_KEY = os.environ.get("STRIPE_API_KEY", "")  # set in your environment if you want real payments
USE_STRIPE = True if STRIPE_API_KEY else False

app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = f"sqlite:///{DATABASE_FILE}"
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
app.config['JWT_SECRET_KEY'] = os.environ.get("JWT_SECRET_KEY", "dev-secret-key")
app.config['PROPAGATE_EXCEPTIONS'] = True

db = SQLAlchemy(app)
jwt = JWTManager(app)

if USE_STRIPE:
    stripe.api_key = STRIPE_API_KEY


# ----------------------
# Models
# ----------------------
class User(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    email = db.Column(db.String(120), unique=True, nullable=False)
    password_hash = db.Column(db.String(256), nullable=False)
    created_at = db.Column(db.DateTime, default=datetime.utcnow)
    is_admin = db.Column(db.Boolean, default=False)

    def set_password(self, password):
        self.password_hash = generate_password_hash(password)

    def check_password(self, password):
        return check_password_hash(self.password_hash, password)


class Product(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    sku = db.Column(db.String(64), unique=True, nullable=False)
    name = db.Column(db.String(200), nullable=False)
    description = db.Column(db.Text, default="")
    price_cents = db.Column(db.Integer, nullable=False)
    inventory = db.Column(db.Integer, default=0)
    created_at = db.Column(db.DateTime, default=datetime.utcnow)


class CartItem(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    user_id = db.Column(db.Integer, db.ForeignKey('user.id'), nullable=False)
    product_id = db.Column(db.Integer, db.ForeignKey('product.id'), nullable=False)
    quantity = db.Column(db.Integer, default=1)
    added_at = db.Column(db.DateTime, default=datetime.utcnow)


class Order(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    user_id = db.Column(db.Integer, db.ForeignKey('user.id'), nullable=False)
    total_cents = db.Column(db.Integer, nullable=False)
    status = db.Column(db.String(50), default="pending")  # pending, paid, failed
    stripe_payment_intent = db.Column(db.String(200), nullable=True)
    created_at = db.Column(db.DateTime, default=datetime.utcnow)


# ----------------------
# Helpers
# ----------------------
def is_admin_user(user_identity):
    u = User.query.filter_by(email=user_identity).first()
    return bool(u and u.is_admin)


# ----------------------
# Routes
# ----------------------

@app.route("/api/ping", methods=["GET"])
def ping():
    return jsonify({"msg": "pong", "time": datetime.utcnow().isoformat()})


# --- AUTH ---
@app.route("/api/auth/signup", methods=["POST"])
def signup():
    data = request.get_json() or {}
    email, password = data.get("email"), data.get("password")
    if not email or not password:
        return jsonify({"msg": "email and password required"}), 400
    if User.query.filter_by(email=email).first():
        return jsonify({"msg": "email already registered"}), 409
    user = User(email=email)
    user.set_password(password)
    user.is_admin = data.get("is_admin", False)
    db.session.add(user)
    db.session.commit()
    return jsonify({"msg": "user created", "email": user.email}), 201


@app.route("/api/auth/login", methods=["POST"])
def login():
    data = request.get_json() or {}
    email, password = data.get("email"), data.get("password")
    if not email or not password:
        return jsonify({"msg": "email and password required"}), 400
    user = User.query.filter_by(email=email).first()
    if not user or not user.check_password(password):
        return jsonify({"msg": "invalid credentials"}), 401
    token = create_access_token(identity=user.email, expires_delta=timedelta(days=7))
    return jsonify({"access_token": token, "user": {"email": user.email, "is_admin": user.is_admin}})


# --- PRODUCTS ---
@app.route("/api/products", methods=["GET"])
def list_products():
    q = request.args.get("q", "").strip()
    page, per_page = int(request.args.get("page", 1)), int(request.args.get("per_page", 20))
    query = Product.query
    if q:
        like = f"%{q}%"
        query = query.filter((Product.name.ilike(like)) | (Product.description.ilike(like)) | (Product.sku.ilike(like)))
    pag = query.order_by(Product.created_at.desc()).paginate(page=page, per_page=per_page, error_out=False)
    items = [{
        "id": p.id, "sku": p.sku, "name": p.name, "description": p.description,
        "price_cents": p.price_cents, "price": p.price_cents / 100.0, "inventory": p.inventory
    } for p in pag.items]
    return jsonify({"products": items, "total": pag.total, "page": page, "per_page": per_page})


@app.route("/api/products/<int:pid>", methods=["GET"])
def get_product(pid):
    p = Product.query.get_or_404(pid)
    return jsonify({
        "id": p.id, "sku": p.sku, "name": p.name, "description": p.description,
        "price_cents": p.price_cents, "price": p.price_cents / 100.0, "inventory": p.inventory
    })


# --- ADMIN: Product CRUD ---
@app.route("/api/admin/products", methods=["POST"])
@jwt_required()
def admin_create_product():
    if not is_admin_user(get_jwt_identity()):
        return jsonify({"msg": "admin required"}), 403
    data = request.get_json() or {}
    sku, name, price = data.get("sku"), data.get("name"), data.get("price")
    inventory, desc = int(data.get("inventory", 0)), data.get("description", "")
    if not sku or not name or price is None:
        return jsonify({"msg": "sku, name, price required"}), 400
    if Product.query.filter_by(sku=sku).first():
        return jsonify({"msg": "sku already exists"}), 409
    p = Product(sku=sku, name=name, description=desc, price_cents=int(round(float(price) * 100)), inventory=inventory)
    db.session.add(p)
    db.session.commit()
    return jsonify({"msg": "product created", "product_id": p.id}), 201


@app.route("/api/admin/products/<int:pid>", methods=["PUT"])
@jwt_required()
def admin_update_product(pid):
    if not is_admin_user(get_jwt_identity()):
        return jsonify({"msg": "admin required"}), 403
    p = Product.query.get_or_404(pid)
    data = request.get_json() or {}
    for field in ("sku", "name", "description", "inventory", "price"):
        if field in data:
            if field == "price":
                p.price_cents = int(round(float(data["price"]) * 100))
            elif field == "inventory":
                p.inventory = int(data["inventory"])
            else:
                setattr(p, field, data[field])
    db.session.commit()
    return jsonify({"msg": "product updated"})


@app.route("/api/admin/products/<int:pid>", methods=["DELETE"])
@jwt_required()
def admin_delete_product(pid):
    if not is_admin_user(get_jwt_identity()):
        return jsonify({"msg": "admin required"}), 403
    p = Product.query.get_or_404(pid)
    db.session.delete(p)
    db.session.commit()
    return jsonify({"msg": "product deleted"})


# --- CART ---
@app.route("/api/cart", methods=["GET"])
@jwt_required()
def view_cart():
    user = User.query.filter_by(email=get_jwt_identity()).first()
    items = CartItem.query.filter_by(user_id=user.id).all()
    out, total_cents = [], 0
    for it in items:
        prod = Product.query.get(it.product_id)
        if not prod:
            continue
        subtotal = prod.price_cents * it.quantity
        total_cents += subtotal
        out.append({
            "cart_item_id": it.id, "product_id": prod.id, "sku": prod.sku, "name": prod.name,
            "price_cents": prod.price_cents, "price": prod.price_cents / 100.0,
            "quantity": it.quantity, "subtotal": subtotal / 100.0
        })
    return jsonify({"items": out, "total_cents": total_cents, "total": total_cents / 100.0})


@app.route("/api/cart", methods=["POST"])
@jwt_required()
def add_to_cart():
    user = User.query.filter_by(email=get_jwt_identity()).first()
    data = request.get_json() or {}
    pid, qty = data.get("product_id"), int(data.get("quantity", 1))
    if not pid:
        return jsonify({"msg": "product_id required"}), 400
    p = Product.query.get(pid)
    if not p:
        return jsonify({"msg": "product not found"}), 404
    if p.inventory < qty:
        return jsonify({"msg": "not enough inventory"}), 400
    ci = CartItem.query.filter_by(user_id=user.id, product_id=pid).first()
    if ci: 
        ci.quantity += qty
    else: 
        db.session.add(CartItem(user_id=user.id, product_id=pid, quantity=qty))
    db.session.commit()
    return jsonify({"msg": "added to cart"})


@app.route("/api/cart/<int:cid>", methods=["DELETE"])
@jwt_required()
def remove_cart_item(cid):
    user = User.query.filter_by(email=get_jwt_identity()).first()
    ci = CartItem.query.filter_by(id=cid, user_id=user.id).first_or_404()
    db.session.delete(ci)
    db.session.commit()
    return jsonify({"msg": "removed"})


# --- CHECKOUT ---
@app.route("/api/checkout", methods=["POST"])
@jwt_required()
def checkout():
    user = User.query.filter_by(email=get_jwt_identity()).first()
    items = CartItem.query.filter_by(user_id=user.id).all()
    if not items:
        return jsonify({"msg": "cart empty"}), 400
    total_cents = sum(Product.query.get(it.product_id).price_cents * it.quantity for it in items if Product.query.get(it.product_id))
    order = Order(user_id=user.id, total_cents=total_cents, status="pending")
    db.session.add(order)
    db.session.commit()

    if USE_STRIPE:
        intent = stripe.PaymentIntent.create(amount=total_cents, currency="usd",
                                             metadata={"order_id": order.id, "user": user.email})
        order.stripe_payment_intent = intent["id"]
        db.session.commit()
        return jsonify({"client_secret": intent["client_secret"], "order_id": order.id, "amount": total_cents / 100.0})
    else:
        order.status = "paid"
        for it in items:
            p = Product.query.get(it.product_id)
            if p: 
                p.inventory -= it.quantity
            db.session.delete(it)
        db.session.commit()
        return jsonify({"msg": "mock payment success", "order_id": order.id, "total": total_cents / 100.0})


# ----------------------
# Utility: init DB
# ----------------------
def init_db(seed=True):
    with app.app_context():
        db.create_all()
        if seed:
            if not User.query.filter_by(email="admin@example.com").first():
                admin = User(email="admin@example.com", is_admin=True)
                admin.set_password("adminpass")
                db.session.add(admin)
            if not User.query.filter_by(email="user@example.com").first():
                user = User(email="user@example.com", is_admin=False)
                user.set_password("userpass")
                db.session.add(user)
            if Product.query.count() == 0:
                db.session.add_all([
                    Product(sku="SKU-RED-001", name="Red T-Shirt", description="Comfortable red t-shirt", price_cents=1999, inventory=50),
                    Product(sku="SKU-BLU-002", name="Blue Jeans", description="Stylish jeans", price_cents=4999, inventory=20),
                    Product(sku="SKU-MUG-003", name="Coffee Mug", description="Ceramic mug", price_cents=1299, inventory=100),
                ])
            db.session.commit()
    print("DB initialized & seeded.")


# ----------------------
# Start server in Jupyter background
# ----------------------
init_db(seed=True)

def run():
    app.run(host="0.0.0.0", port=5000, debug=True, use_reloader=False)

t = Thread(target=run)
t.start()


DB initialized & seeded.
 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://10.251.190.119:5000
Press CTRL+C to quit
